In [370]:
import numpy as np
import pandas as pd
from numpy import genfromtxt
import matplotlib.pyplot as plt

In [371]:
# load dataset
data = pd.read_csv('Ozone Level Detection.data',
                   sep=";|:|,",
                   header=None,
                   engine='python')

# change missing value to np.nan
data[data == '?'] = np.nan

#data.isnull().sum()
np_data = data.to_numpy()
#data

#رفع داده های گم‌شده

برای رفع داده های گم شده از منبع زیر استفاده شده است.
https://www.analyticsvidhya.com/blog/2021/10/handling-missing-value/


Missing values are imputed using the k-Nearest Neighbors approach where a Euclidean distance is used to find the nearest neighbors.

In [372]:
X = data.iloc[:,1:74]

In [373]:
from sklearn.impute import KNNImputer
impute_knn = KNNImputer(n_neighbors=5)
FixedInputData = impute_knn.fit_transform(X)
FixedInputData = pd.DataFrame(FixedInputData)
#FixedInputData.insert(0,-1, np_data[:,0]) Adding date as property
FixedInputData = FixedInputData.to_numpy()



## implementing KFold

ورودی این تابع تعداد تقسیم، آرایه و اینکه رندوم بشود یا نه را می گیرد و خروجی آن آرایه ای از کلاس فولد می دهد که شامل داده ی آموزش و تست است. بدیهی است که تعداد این آرایه برابر تعداد تقسیم که در ورودی گرفته است، می باشد.

In [374]:
class Fold:
    def __init__(self,X_Train,Y_Train,X_Test,Y_Test):
        self.X_Train = X_Train
        self.Y_Train = Y_Train
        self.X_Test = X_Test
        self.Y_Test = Y_Test


def KFold(NumberOfSplits, InputData, Shuffle=False):
    if(Shuffle):
         np.random.shuffle(InputData)
    output = []

    SplitedArray = np.array_split(InputData, NumberOfSplits)
    for i in range(NumberOfSplits):
        test = SplitedArray[i]
        train = np.concatenate( SplitedArray[0:i] +  SplitedArray[i + 1 :], axis=0 )
        X_Train = train[:,:-1]
        Y_Train = train[:, -1]

        X_Test = test[:,:-1]
        Y_Test = test[:,-1]
        output.append(Fold(X_Train,Y_Train,X_Test,Y_Test))
    
    return output
 

    

In [375]:
kfold_Out = KFold(3,FixedInputData)
kfold_Out[0].X_Train.shape[0]

1690

In [376]:
def sigmoid(x):
    return (1/(1 +  np.e ** (-x)))

In [377]:
def CrossEntropyLossFunction(M, Y_Hat, Y):
    return (-1/M) * (Y.dot( np.log(Y_Hat).T) + (1-Y).dot( np.log(1 - Y_Hat).T))

In [378]:
def CalculateYHat(W,X,b):
    return np.dot(W , np.transpose(X)) + b

In [392]:
nr_split = 3
kfold_Out = KFold(nr_split,FixedInputData)
kfold_Out[0].X_Train.shape[0]


learningRate = 0.000001
nr_epoch = 500

for i in range (1): #TODO
    # for each split do the training
    X = kfold_Out[i].X_Train
    Y = np.reshape( kfold_Out[i].Y_Train,(1, kfold_Out[i].Y_Train.shape[0]))
    M = X.shape[0]
    W = np.zeros((1 ,X.shape[1])) # 1 * 72
    b = np.zeros((1,1)) # 1 * 1

    for j in range(nr_epoch): # traning for nr_epoch
            
        Z = CalculateYHat(W,X,b)  # W => 1 * 72 ||||| X => M * 72 
        
        A = sigmoid(Z) # Y_hat
        
        L = CrossEntropyLossFunction(M,A, Y)
        
        # dL/dZ
        dZ = A - Y 
        print("i:" , j , " loss:",L)
        #print(Z)
        # Calculate derivitives
        dW = (1/M) * (np.dot(X.T,dZ.T)) # dL/dW
        
        db = (1/M) * np.sum(dZ) # dL/db

        # update the weights and bias
        W = W - learningRate * np.transpose(dW)
        
        b = b - learningRate * np.transpose(db)
        
        


    # Calculate loss for current split
    




i: 0  loss: [[0.69314718]]
i: 1  loss: [[1.76838483]]
i: 2  loss: [[1.67570469]]
i: 3  loss: [[1.58302455]]
i: 4  loss: [[1.49034441]]
i: 5  loss: [[1.39766427]]
i: 6  loss: [[1.30498413]]
i: 7  loss: [[1.21230399]]
i: 8  loss: [[1.11962385]]
i: 9  loss: [[1.02694371]]
i: 10  loss: [[0.93426357]]
i: 11  loss: [[0.84158343]]
i: 12  loss: [[0.74890329]]
i: 13  loss: [[0.65622315]]
i: 14  loss: [[0.56354301]]
i: 15  loss: [[0.47086288]]
i: 16  loss: [[0.37818301]]
i: 17  loss: [[0.28551403]]
i: 18  loss: [[0.19329432]]
i: 19  loss: [[0.11838901]]
i: 20  loss: [[0.12778445]]
i: 21  loss: [[0.18592147]]
i: 22  loss: [[0.1167162]]
i: 23  loss: [[0.11807003]]
i: 24  loss: [[0.12755242]]
i: 25  loss: [[0.15278211]]
i: 26  loss: [[0.30384197]]
i: 27  loss: [[0.21138231]]
i: 28  loss: [[0.12751934]]
i: 29  loss: [[0.15275217]]
i: 30  loss: [[0.30375206]]
i: 31  loss: [[0.2112931]]
i: 32  loss: [[0.12745697]]
i: 33  loss: [[0.15269636]]
i: 34  loss: [[0.30355145]]
i: 35  loss: [[0.21109417]]
i: 3